#### The project is the hangman game, the player has to guess a letter from a random word or against a player.
#### This first player has 5 attempts to guess the word, if he guessed a letter correctly, win a point. 
#### If he guessed the word, win 10 points, if  he guessed a wrong letter, lose a point.
#### If he did not guess the word, lose 10 points.
#### There is a 2 player mode, where one player write the word and the other one try to guess.
#### In case the player plays against the PC, the PC chooses a random word from a list.

## 1) Imports and transform words from archive

In [ ]:
#import numpy
import numpy as np

# dataframe
import pandas as pd

# regular expression
import re

# remove accents
import unidecode

# random
import random

#mySQL connect
import mysql.connector

# system
import sys

# warnings
import warnings
warnings.filterwarnings('ignore')

#### 1.1) Export and Load words archive

In [ ]:
f = open("C:/Users/Aluno/Desktop/UPSKILL/Trabalhos/M1 - Trabalho Intro.prog/p.txt", 'r')
l1 = f.readlines()
print(len(l1))
print((type(l1)))

#### 1.2) Transform words

In [ ]:
# create a dataframe
df = pd.read_csv('C:/Users/Aluno/Desktop/UPSKILL/Trabalhos/M1 - Trabalho Intro.prog/p.txt', header=None, encoding = "ISO-8859-1")
print(df.head(5))
print(df.tail(5))

In [ ]:
# name dataframe column 0 as "words"
words = df[0]

In [ ]:
print(words)

In [ ]:
# with regular expression remove acronyms, names like Aachen and Aalborg
pattern = "^[^A-Z]{2}[^Aa]"
df1 = words[words.str.contains(pattern)]
print(df1)

In [ ]:
# create a new dataframe with words ONLY with hyphen
pattern = "[\w]+[-]"
df2 = df1[words.str.contains(pattern)]
print(df2)

In [ ]:
# remove words from df1 with hyphen and call it df3
df3 = pd.concat([df1, df2, df2]).drop_duplicates(keep=False)

In [ ]:
# transforming the dataframe into a list without accents
flist = df3.values.tolist()
unaccented_list = []
for i in range (len(flist)):
    decode = unidecode.unidecode(flist[i])
    unaccented_list.append(decode)

## 2) Open DB connection

In [ ]:
con = mysql.connector.connect(
      host="localhost",
      user="root",
      passwd="test",
      #passwd="",
      database="dbgame"
    )
mycursor = con.cursor()

## 3) Insert values into MySQL DB

In [ ]:
# insert MySQL values
def insertdb():
    qry = f"""INSERT INTO dbgame.game (name, points) VALUES ('{name}', '{total_points}') """
    mycursor.execute(qry)
    con.commit()

## 4) Retrieve previous points from MySQL DB

In [ ]:
def retrieve(name):
    qry = f""" SELECT * FROM dbgame.game WHERE name = '{name}' ORDER BY id DESC LIMIT 1 """
    mycursor.execute(qry)
    records = mycursor.fetchall()
    for row in records:
        name = row[1]
        a = row[2] 
        if name == name:
            a = row[2]
            print("Previous points: ", a)
            break
        else:
            a = 0

## 5) The game

In [ ]:
name = input("Write your name: ")
total_points = 0

# Function to transform word into a list without accents

def unaccented(word):
    unaccented_list = []
    for i in range(len(word)):
        decode = unidecode.unidecode(word[i])
        unaccented_list.append(decode)
    return unaccented_list
    
# player vs player
def player(word):    
    return unaccented(word)   

# menu
def menu():
    print("[1] Player vs Player.")
    print("[2] Player against a random word from computer.")
    print("[3] My points.")
    print("[0] Exit")
    print("\n")

# according to the option, get word
def get_word():    
    menu()
    global name
    end = False
    x = int(input("Enter the option: "))
    while x != 0 and end == False:     
        if x == 1:
            word = list(input("Enter the word to play: ").upper()) 
            return player(word)  
        if x == 2:
            word = list(random.choice(unaccented_list).upper())
            return word        
        if x == 3:           
            retrieve(name)
            end = True            
        else:
            print("Invalid option. Choose again. \n")
            menu()
            x = int(input("Enter the option: "))   
    sys.exit("Bye.")        
        
# Play game:
def play(word):
    word2 = list(word)
    choice = []
    wrong = []
    right = []
    points = 0
    attempts = 0
    max_attempts = 5    
    global name
    global total_points 
    
    # get previous points from player
    qry = f""" SELECT * FROM dbgame.game WHERE name = '{name}' ORDER BY id DESC LIMIT 1 """
    mycursor.execute(qry)
    records = mycursor.fetchall()
    for row in records:
        name = row[1]
        a = row[2]  
        if name == name:
            a = row[2]
            break
    else:
        a = 0    

#loop through the word and create "_"
    for i in word:
        choice.append('_')

# loop until the player has won or lost
    end = False
    while not end:
    
    # display attempts, letters, hangman and points
        print('{} attempts'.format(max_attempts - attempts))
        print('The current word is: {} '.format(' '.join(choice)))
        print('     ------')
        print('     |    |')
        print('     |    ' + ('O' if attempts >= 1 else ''))
        print('     |    ' + ('/\\' if attempts >= 2 else ''))
        print('     |    ' + ('|' if attempts >= 3 else ''))
        print('     |    ' + ('/\\' if attempts >= 4 else ''))
        print('   ------')
    
    # Ask the player to guess
        guess = (input("Guess a letter: ")).upper()          
        print('\n\n\n')
    
    #If the player guessed right, show all matched letters and print it
        if guess in word:
            print("Guessed correctly the {} in the word".format(guess))
            points += 1
            print("points: ", points)
            right.append(guess)
            for i in range(len(word)):
                char = word[i]
                if char == guess:            
                    choice[i] = word[i]
                    word[i] = "_"
                    
        elif guess not in word and guess not in right and guess not in wrong:
            print("Guessed wrongly the {} in the word".format(guess))
            points -= 1
            attempts += 1
            wrong.append(guess)
            print("points: ", points)
                    
        elif guess in right:
            print("You already guessed this word!")
            
        elif guess in wrong:
            print("You already guessed this word!")                
        
        # if the player has won print a win message and stop looping
        if all("_" == char for char in word):
            print("Congrats, you won!")
            total_points = 10 + a + points
            print("Total points: ", total_points)
            print("Word", word2)
            end = True
            
        # if the player has lost, print failing and stop looping
        if attempts >= max_attempts:
            print("Word: ", (word2))
            print("You lost!")
            total_points = -10 + a + points     
            print("Total points: ", total_points)
            end = True 

def main():
    word = get_word()
    play(word)    
    insertdb()        
    while input("Play Again? (Y/N) ").upper() == "Y":
        word = get_word()
        play(word)        
        insertdb()                
    else:
        print("THE END...")        

if __name__ == "__main__":
    main()